# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [17]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference. I should search for venues that can accommodate the event's theme, size, and budget constraints.

Action: Search the internet
Action Input: {"search_query": "venues for events in San Francisco"} 


Search results: Title: Top 20 Venues in San Francisco, CA - PartySlate
Link: https://www.partyslate.com/find-venues/near/san-francisco-ca-usa
Snippet: Top San Francisco Venues ; The Westin St. Francis. 335 Powell St, San Francisco, CA 94102, USA. Spaces for 60-1,500 Guests. 8 Hotels. 8 Ballrooms ; Pier 27. The ...
---
Title: Party Venues near San Francisco, CA - Eventective
Link: https://www.eventective.com/san-francisco-ca/party-event-venues/
Snippet: There are 417 Party Venues in and around San F

Final Answer: 
Based on the search results, the Tech Innovation Conference is scheduled to take place on September 10-12, 2024, at The Westin St. Francis in San Francisco, USA. Unfortunately, the search did not provide specific pricing information for the event at this venue.

> Finished chain.
 

Based on the search results, the Tech Innovation Conference is scheduled to take place on September 10-12, 2024, at The Westin St. Francis in San Francisco, USA. Unfortunately, the search did not provide specific pricing information for the event at this venue.

Thought: 
I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference. I should search for venues that can accommodate the event's theme, size, and budget constraints.

Action: Read website content
Action Input: {"website_url": "https://www.partyslate.com/find-venues/near/san-francisco-ca-usa"} 

Top Venues in San Francisco, CA | PartySlate

This is the agent final answer: After researching poten

Final Answer:
After searching for availability and pricing information for The Westin St. Francis and Pier 27 for the dates of September 10-12, 2024, I was unable to find specific details on pricing and availability for those dates. Additional research or direct contact with the venues may be necessary to obtain this information.

> Finished chain.
 

After searching for availability and pricing information for The Westin St. Francis and Pier 27 for the dates of September 10-12, 2024, I was unable to find specific details on pricing and availability for those dates. Additional research or direct contact with the venues may be necessary to obtain this information.

Final Answer: 
Further research is needed to determine the availability and pricing of The Westin St. Francis and Pier 27 in San Francisco for the dates of September 10-12, 2024, for the Tech Innovation Conference. Additional steps may include contacting the venues directly to gather more detailed information on their availab

This is the agent final answer: Attention Required! | Cloudflare Please enable cookies. Sorry, you have been blocked You are unable to access marriott.com Why have I been blocked? This website is using a security service to protect itself from online attacks. The action you just performed triggered the security solution. There are several actions that could trigger this block including submitting a certain word or phrase, a SQL command or malformed data. What can I do to resolve this? You can email the site owner to let them know you were blocked. Please include what you were doing when this page came up and the Cloudflare Ray ID found at the bottom of this page. Cloudflare Ray ID: 8b6cbffb8b9417f2 • Your IP: Click to reveal 52.52.221.170 • Performance & security by Cloudflare
Please provide a feedback: 

Exception in thread Thread-14 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/u

Final Answer: 
# Marketing Activities and Attendee Engagement Report

## Marketing Strategies Found:
- Explored various conference marketing ideas for 2024 that really work, including virtual event tips, to boost engagement and success.
- Identified the benefits of conference marketing in creating a community, fostering interactions, reaching the right audience, increasing brand awareness, showcasing authority, generating buzz on social media, and securing partnerships/sponsorships.
- Highlighted the importance of pre-marketing essentials such as identifying target audience, segmenting audiences, and optimizing the event website.

## Potential Marketing Activities:
1. Craft personalized emails for increased subscriber engagement.
2. Encourage speakers to promote their involvement.
3. Form influencer partnerships for niche audience reach.
4. Maximize reach with social media engagement.
5. Offer incentives to increase attendance.
6. Drive engagement with video marketing.
7. Host live bro

- Display the generated `venue_details.json` file.

In [15]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '135 Fourth Street, 4th Floor, San Francisco, CA 94103',
 'booking_status': 'Available',
 'capacity': 300,
 'name': 'City View at METREON'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [16]:
from IPython.display import Markdown
Markdown("marketing_report.md")

marketing_report.md